# Getting Connected

The simplest way to try out a connection to the Neo4j server is via the console. Once you have started a local Neo4j server, open a new Python console and enter the following:

In [1]:
from py2neo import Graph
graph = Graph()

# Nodes & Relationships

Nodes and relationships are the fundamental building blocks of a Neo4j graph and both have a corresponding class in py2neo. Assuming we’ve already established a connection to the server (as above) let’s build a simple graph with two nodes and one relationship:

In [2]:
from py2neo import Node, Relationship
alice = Node("Person", name="Alice")
bob = Node("Person", name="Bob")
alice_knows_bob = Relationship(alice, "KNOWS", bob)
graph.create(alice_knows_bob)

(<Relationship graph=u'http://localhost:7474/db/data/' ref=u'relationship/2' start=u'node/3' end=u'node/4' type=u'KNOWS' properties={}>,)

When first created, Node and Relationship objects exist only in the client; nothing has yet been written to the server. The Graph.create method shown above creates corresponding server-side objects and automatically binds each local object to its remote counterpart. 

Within py2neo, binding is the process of applying a URI to a client object thereby allowing future client-server synchonisation operations to occur.

# Pushing & Pulling

Client-server communication over REST can be chatty if not used carefully. Whenever possible, py2neo attempts to minimise the amount of chatter between the client and the server by batching and lazily retrieving data. Most read and write operations are explicit, allowing the Python application developer a high degree of control over network traffic.

To illustrate synchronisation, let’s give Alice and Bob an age property each. Longhand, this is done as follows:

In [3]:
alice.properties["age"] = 33
bob.properties["age"] = 44
alice.push()
bob.push()

Here, we add a new property to each of the two local nodes and push the changes in turn. This results in two separate HTTP calls being made to the server which can be seen more clearly with the debugging function, watch:

To squash these two separate push operations into one, we can use the Graph.push method instead:

In [4]:
graph.push(alice, bob)

# Cypher

## Single Statements

Neo4j has a built-in data query and manipulation language called Cypher. To execute Cypher from within py2neo, simply use the cypher attribute of a Graph instance and call the execute method:

In [5]:
graph.cypher.execute("MERGE (c:Person {name:{N}}) RETURN c", {"N": "Carol"})

   | c                         
---+----------------------------
 1 | (n5:Person {name:"Carol"})

The object returned from this call is a RecordList which is rendered by default as a table of results. Each item in this list is a Record instance:

In [6]:
for record in graph.cypher.execute("CREATE (d:Person {name:'Dave'}) RETURN d"):
    print(record)

 d                        
---------------------------
 (n6:Person {name:"Dave"})



A Record exposes its values through both named attributes and numeric indexes. Therefore, if a Cypher query returns a column called name, that column can be accessed through the record attribute called name:

In [7]:
for record in graph.cypher.execute("MATCH (p:Person) RETURN p.name AS name"):
    print(record.name)

Ann
Dan
Alice
Alice
Bob
Carol
Dave


Similarly, the first column returned can be accessed as column 0:

In [8]:
for record in graph.cypher.execute("MATCH (p:Person) RETURN p.name AS name"):
    print(record[0])

Ann
Dan
Alice
Alice
Bob
Carol
Dave


Transactions
http://py2neo.org/2.0/intro.html